<a href="https://colab.research.google.com/github/Formula-Electric-Berkeley/FEBSim/blob/main/Steering_Effort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load and clean the data
data_path = 'B1320run52.csv'
data = pd.read_csv(data_path)
data = data.drop(data.index[0])
data['FY'] = pd.to_numeric(data['FY'], errors='coerce')
data['SA'] = pd.to_numeric(data['SA'], errors='coerce')
data['FZ'] = pd.to_numeric(data['FZ'], errors='coerce')
data['IA'] = pd.to_numeric(data['IA'], errors='coerce')
data['V'] = pd.to_numeric(data['V'], errors='coerce')
data['MZ'] = pd.to_numeric(data['MZ'], errors='coerce') # Assuming MZ is also a column
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(subset=['FY', 'SA', 'FZ', 'IA', 'V', 'MZ'], inplace=True)

# Constants
mechanical_trail = 0.006
upright_steering_arm = 0.05
steering_rack_trail = 0.01
rack_ratio = 3.36

# Define bins with specific ranges
bins = [
    {'name': 'Outer', 'fz_range': (-1200, -1000), 'ia_range': (0.9, 1.1), 'v_range': (0, 42)},
    {'name': 'Inner', 'fz_range': (-300, -100), 'ia_range': (2.9, 3.1), 'v_range': (0, 42)}
]

# Prepare to plot
fig, ax1 = plt.subplots(figsize=(12, 6))

# Custom labels for the curves
custom_labels = {
    'Outer': 'Outer Normal Load (1100 N and 1.0 degree camber)',
    'Inner': 'Inner Normal Load (200 N and 3.0 degree camber)'
}

# Process each bin
combined_steering_torque = None

for bin_info in bins:
    name = bin_info['name']
    filtered_data = data[
        (data['FZ'] >= bin_info['fz_range'][0]) & (data['FZ'] <= bin_info['fz_range'][1]) &
        (data['IA'] >= bin_info['ia_range'][0]) & (data['IA'] <= bin_info['ia_range'][1]) &
        (data['V'] >= bin_info['v_range'][0]) & (data['V'] <= bin_info['v_range'][1])
    ].copy()

    if not filtered_data.empty:
        # Calculate Steering Torque
        filtered_data[f'SteeringTorque{name}'] = (filtered_data['MZ'] + (filtered_data['FY'] * mechanical_trail)) / upright_steering_arm * (steering_rack_trail / rack_ratio)

        # Plot MZ vs Slip Angle for this bin
        ax1.plot(filtered_data['SA'], filtered_data['MZ'], label=f'MZ - {custom_labels[name]}')

        if combined_steering_torque is None:
            combined_steering_torque = filtered_data[['SA', f'SteeringTorque{name}']].copy()
            combined_steering_torque.rename(columns={f'SteeringTorque{name}': 'CombinedSteeringTorque'}, inplace=True)
        else:
            combined_steering_torque = combined_steering_torque.merge(filtered_data[['SA', f'SteeringTorque{name}']], on='SA', how='inner')
            combined_steering_torque['CombinedSteeringTorque'] += combined_steering_torque[f'SteeringTorque{name}']

# Plot combined steering torque
if combined_steering_torque is not None:
    ax1.plot(combined_steering_torque['SA'], combined_steering_torque['CombinedSteeringTorque'], color='green', label='Combined Steering Torque')

# Set axis titles and limits
ax1.set_title('Steering Torque and MZ vs Slip Angle')
ax1.set_xlabel('Slip Angle (deg)')
ax1.set_ylabel('MZ and Steering Torque (Nm)')
ax1.set_ylim(-100, 100)

ax2 = ax1.twinx()
ax2.set_ylabel('Lateral Force (N)')
ax2.set_ylim(-2000, 2000)

# Display the legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Show the plot
plt.show()
